In [2]:
!pip install -q torchvision==0.15.2 torch==2.2.0 albumentations==1.3.1

import os, zipfile

ZIP_PATH = "/content/Labeled Data.zip"
DATA_ROOT = "/content/data"

if not os.path.exists(DATA_ROOT):
    with zipfile.ZipFile(ZIP_PATH, 'r') as z:
        z.extractall(DATA_ROOT)

print("Folders:", os.listdir(DATA_ROOT))


ERROR: Ignored the following yanked versions: 0.1.6, 0.1.7, 0.1.8, 0.1.9, 0.2.0, 0.2.1, 0.2.2, 0.2.2.post2, 0.2.2.post3
ERROR: Could not find a version that satisfies the requirement torchvision==0.15.2 (from versions: 0.17.0, 0.17.1, 0.17.2, 0.18.0, 0.18.1, 0.19.0, 0.19.1, 0.20.0, 0.20.1, 0.21.0, 0.22.0, 0.22.1, 0.23.0, 0.24.0, 0.24.1)
ERROR: No matching distribution found for torchvision==0.15.2
Folders: ['Labeled Data']


In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
import os
from PIL import Image
import random
import numpy as np
import cv2

# Histogram Equalization for low quality faces
class Equalize:
    def __call__(self, img):
        img_np = np.array(img)

        # Convert to grayscale
        gray = cv2.cvtColor(img_np, cv2.COLOR_RGB2GRAY)

        # Apply histogram equalization
        eq = cv2.equalizeHist(gray)

        # Convert back to RGB (3 channels)
        eq_rgb = cv2.cvtColor(eq, cv2.COLOR_GRAY2RGB)

        return Image.fromarray(eq_rgb)



In [4]:
root_dir = "/content/data/Labeled Data"  # folder that contains student folders

all_train_files = []
all_val_files = []

for student in sorted(os.listdir(root_dir)):
    student_path = os.path.join(root_dir, student)
    if not os.path.isdir(student_path):
        continue

    images = [os.path.join(student_path, x)
              for x in os.listdir(student_path)
              if x.lower().endswith(("jpg", "png", "jpeg"))]

    if len(images) == 0:
        continue
    elif len(images) == 1:
        # ONLY ONE IMAGE → training only
        all_train_files.extend([(images[0], student)])
    else:
        labels = [student] * len(images)
        train_imgs, val_imgs = train_test_split(
            images, test_size=0.2, random_state=42
        )
        all_train_files.extend([(x, student) for x in train_imgs])
        all_val_files.extend([(x, student) for x in val_imgs])

print("Train size:", len(all_train_files))
print("Val size:", len(all_val_files))


Train size: 876
Val size: 232


In [5]:
class FaceDataset(Dataset):
    def __init__(self, file_list, class_to_idx, transform=None):
        self.file_list = file_list
        self.class_to_idx = class_to_idx
        self.transform = transform

    def __getitem__(self, idx):
        img_path, label_name = self.file_list[idx]
        img = Image.open(img_path).convert("RGB")
        if self.transform:
            img = self.transform(img)
        return img, self.class_to_idx[label_name]

    def __len__(self):
        return len(self.file_list)


In [6]:
train_transform = transforms.Compose([
    Equalize(),
    transforms.Resize((224,224)),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.4, contrast=0.4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

val_transform = transforms.Compose([
    Equalize(),
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])


In [7]:
# extract class names
class_names = sorted(os.listdir(root_dir))
class_to_idx = {c:i for i,c in enumerate(class_names)}

train_ds = FaceDataset(all_train_files, class_to_idx, train_transform)
val_ds   = FaceDataset(all_val_files,   class_to_idx, val_transform)

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
val_loader   = DataLoader(val_ds, batch_size=32, shuffle=False)

len(train_ds), len(val_ds)


(876, 232)

In [8]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model = models.resnet18(weights="IMAGENET1K_V1")
model.fc = nn.Linear(model.fc.in_features, len(class_names))
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)


In [9]:
def train_one_epoch():
    model.train()
    total_loss = 0
    correct = 0

    for imgs, labels in train_loader:
        imgs, labels = imgs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(imgs)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        correct += (outputs.argmax(1) == labels).sum().item()

    return total_loss / len(train_loader), correct / len(train_ds)


def validate():
    model.eval()
    total_loss = 0
    correct = 0

    with torch.no_grad():
        for imgs, labels in val_loader:
            imgs, labels = imgs.to(device), labels.to(device)

            outputs = model(imgs)
            loss = criterion(outputs, labels)

            total_loss += loss.item()
            correct += (outputs.argmax(1) == labels).sum().item()

    return total_loss / len(val_loader), correct / len(val_ds)

# TRAIN
for epoch in range(15):
    train_loss, train_acc = train_one_epoch()
    val_loss, val_acc = validate()
    print(f"Epoch {epoch+1}: "
          f"Train Loss={train_loss:.4f}, Acc={train_acc:.3f} | "
          f"Val Loss={val_loss:.4f}, Acc={val_acc:.3f}")


Epoch 1: Train Loss=2.6357, Acc=0.368 | Val Loss=1.7102, Acc=0.638
Epoch 2: Train Loss=1.0194, Acc=0.882 | Val Loss=0.6123, Acc=0.957
Epoch 3: Train Loss=0.3876, Acc=0.976 | Val Loss=0.2596, Acc=0.978
Epoch 4: Train Loss=0.2009, Acc=0.989 | Val Loss=0.1515, Acc=0.983
Epoch 5: Train Loss=0.1097, Acc=0.997 | Val Loss=0.1212, Acc=0.978
Epoch 6: Train Loss=0.0839, Acc=0.997 | Val Loss=0.0891, Acc=0.991
Epoch 7: Train Loss=0.0596, Acc=0.998 | Val Loss=0.0897, Acc=0.987
Epoch 8: Train Loss=0.0439, Acc=0.999 | Val Loss=0.0716, Acc=0.991
Epoch 9: Train Loss=0.0338, Acc=0.999 | Val Loss=0.0685, Acc=0.987
Epoch 10: Train Loss=0.0296, Acc=0.999 | Val Loss=0.0651, Acc=0.991
Epoch 11: Train Loss=0.0244, Acc=0.999 | Val Loss=0.0555, Acc=0.991
Epoch 12: Train Loss=0.0186, Acc=1.000 | Val Loss=0.0537, Acc=0.991
Epoch 13: Train Loss=0.0160, Acc=1.000 | Val Loss=0.0537, Acc=0.991
Epoch 14: Train Loss=0.0148, Acc=1.000 | Val Loss=0.0517, Acc=0.991
Epoch 15: Train Loss=0.0124, Acc=1.000 | Val Loss=0.0480,

In [10]:
torch.save(model.state_dict(), "/content/face_recognition_model.pth")


In [13]:
# === Load model for testing ===
import torch
from PIL import Image
from torchvision import transforms

device = "cuda" if torch.cuda.is_available() else "cpu"

# Load model
model = models.resnet18(weights="IMAGENET1K_V1")
model.fc = nn.Linear(model.fc.in_features, len(class_names))
model.load_state_dict(torch.load("/content/face_recognition_model.pth", map_location=device))
model = model.to(device)
model.eval()

# SAME validation transform
test_transform = transforms.Compose([
    Equalize(),
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

def predict_image(img_path):
    img = Image.open(img_path).convert("RGB")
    img_tensor = test_transform(img).unsqueeze(0).to(device)

    with torch.no_grad():
        preds = model(img_tensor)
        class_id = preds.argmax(1).item()

    return class_names[class_id]


# === Upload and predict ===
from google.colab import files
uploaded = files.upload()

test_img = list(uploaded.keys())[0]
print("\nPredicting:", test_img)

pred_class = predict_image(test_img)
print("\n🔥 Predicted Class:", pred_class)


Saving face_007_conf_1.00 (2).jpg to face_007_conf_1.00 (2).jpg

Predicting: face_007_conf_1.00 (2).jpg

🔥 Predicted Class: Habiba Ayman


In [16]:

from google.colab import files
uploaded = files.upload()

test_img = list(uploaded.keys())[0]
print("\nPredicting:", test_img)

pred_class = predict_image(test_img)
print("\n🔥 Predicted Class:", pred_class)


Saving face_004_conf_1.00 (2).jpg to face_004_conf_1.00 (2).jpg

Predicting: face_004_conf_1.00 (2).jpg

🔥 Predicted Class: Hania Khaled


In [17]:

from google.colab import files
uploaded = files.upload()

test_img = list(uploaded.keys())[0]
print("\nPredicting:", test_img)

pred_class = predict_image(test_img)
print("\n🔥 Predicted Class:", pred_class)


Saving face_013_conf_1.00 (2).jpg to face_013_conf_1.00 (2).jpg

Predicting: face_013_conf_1.00 (2).jpg

🔥 Predicted Class: Ossama


In [21]:
# ===== CREATE A TEST SET FROM VAL IMAGES =====
test_dir = "/content/test_set"
os.makedirs(test_dir, exist_ok=True)

# Copy validation images to test folder (optional)
import shutil

for path, label in all_val_files:
    student_folder = os.path.join(test_dir, label)
    os.makedirs(student_folder, exist_ok=True)
    shutil.copy(path, student_folder)

print("Test Folder Created With:", len(all_val_files), "images")


Test Folder Created With: 232 images


In [22]:
def evaluate_test_set(test_root):
    model.eval()
    correct = 0
    total = 0

    for student in sorted(os.listdir(test_root)):
        student_folder = os.path.join(test_root, student)
        if not os.path.isdir(student_folder):
            continue

        for img_file in os.listdir(student_folder):
            if img_file.lower().endswith(("jpg", "jpeg", "png")):
                img_path = os.path.join(student_folder, img_file)
                pred = predict_image(img_path)

                total += 1
                if pred == student:
                    correct += 1

                print(f"{img_file} → Predicted: {pred} | True: {student}")

    accuracy = correct / total if total > 0 else 0
    print("\n===============================")
    print(f"🔥 FINAL TEST ACCURACY: {accuracy*100:.2f}%")
    print("===============================")


In [23]:
evaluate_test_set("/content/test_set")


face_006_conf_1.00 (4).jpg → Predicted: Abdallah Tamer | True: Abdallah Tamer
face_005_conf_1.00 (2).jpg → Predicted: Abdallah Tamer | True: Abdallah Tamer
face_004_conf_1.00.jpg → Predicted: Abdallah Tamer | True: Abdallah Tamer
face_013_conf_1.00.jpg → Predicted: Abdallah Tamer | True: Abdallah Tamer
face_009_conf_1.00 (3).jpg → Predicted: Abdallah Tamer | True: Abdallah Tamer
face_002_conf_1.00.jpg → Predicted: Abdallah Tamer | True: Abdallah Tamer
face_009_conf_1.00.jpg → Predicted: Abdallah Tamer | True: Abdallah Tamer
face_012_conf_1.00 (3).jpg → Predicted: Abdallah Tamer | True: Abdallah Tamer
face_004_conf_1.00 (2).jpg → Predicted: Abdallah Tamer | True: Abdallah Tamer
face_008_conf_1.00 (4).jpg → Predicted: Abdallah Tamer | True: Abdallah Tamer
face_017_conf_1.00.jpg → Predicted: Ahmed Amer | True: Ahmed Amer
face_002_conf_1.00.jpg → Predicted: Ahmed Amer | True: Ahmed Amer
face_015_conf_1.00.jpg → Predicted: Ahmed Amer | True: Ahmed Amer
face_018_conf_1.00.jpg → Predicted: Am